In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F

In [24]:
from sklearn.datasets import load_diabetes

dataset = load_diabetes(scaled=True)
X, y = dataset['data'], dataset['target']

In [36]:
from torch.utils.data import DataLoader, Dataset, random_split

class DiabetesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

full_dataset = DiabetesDataset(X, y)

train_size = int(len(X) * 0.8)
test_size = len(X) - train_size

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [59]:
def r2_score(y_pred, y_true):
    ss_res = torch.sum((y_true - y_pred) ** 2)
    ss_tot = torch.sum((y_true - torch.mean(y_true)) ** 2)
    return 1 - ss_res / ss_tot

In [70]:
class MLP(nn.Module):

    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, num_hidden), nn.ReLU(),
            nn.Linear(num_hidden, num_outputs)
        )

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [71]:
model = MLP(X.shape[1], 10, 1)
criterion = nn.MSELoss()
optim = torch.optim.SGD(model.parameters(), lr=0.001)

In [73]:
for epoch in range(100):
    model.train()
    train_loss, train_r2 = 0, 0
    for X_batch, y_batch in train_loader:

        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)

        optim.zero_grad()
        loss.backward()
        optim.step()

        train_loss += loss.item()
        train_r2 += r2_score(y_pred, y_batch).item()
    
    train_loss /= len(train_loader)
    train_r2 /= len(train_loader)

    model.eval()
    test_loss, test_r2 = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)

            test_loss += loss.item()
            test_r2 += r2_score(y_pred, y_batch).item()
        
        test_loss /= len(test_loader)
        test_r2 /= len(test_loader)

    print(f'Epoch {epoch} | Train Loss: {train_loss ** 0.5:.4f} | R2: {train_r2:.4f} | Test Loss: {test_loss ** 0.5:.4f} | R2 {test_r2:.4g}')

Epoch 0 | Train Loss: 58.6141 | R2: -inf | Test Loss: 66.3251 | R2 -0.0004635
Epoch 1 | Train Loss: 56.7045 | R2: -inf | Test Loss: 62.4993 | R2 0.1182
Epoch 2 | Train Loss: 56.6191 | R2: -inf | Test Loss: 57.5709 | R2 0.2554
Epoch 3 | Train Loss: 55.4683 | R2: -inf | Test Loss: 82.0527 | R2 -0.5745
Epoch 4 | Train Loss: 62.0880 | R2: -inf | Test Loss: 115.4230 | R2 -2.163
Epoch 5 | Train Loss: 66.2551 | R2: -inf | Test Loss: 122.0380 | R2 -2.541
Epoch 6 | Train Loss: 64.0685 | R2: -inf | Test Loss: 88.7397 | R2 -0.8495
Epoch 7 | Train Loss: 56.4598 | R2: -inf | Test Loss: 57.6098 | R2 0.2566
Epoch 8 | Train Loss: 61.8367 | R2: -inf | Test Loss: 84.6800 | R2 -0.6531
Epoch 9 | Train Loss: 64.1870 | R2: -inf | Test Loss: 70.6724 | R2 -0.1422
Epoch 10 | Train Loss: 61.0728 | R2: -inf | Test Loss: 94.1871 | R2 -1.088
Epoch 11 | Train Loss: 62.5397 | R2: -inf | Test Loss: 76.6594 | R2 -0.3358
Epoch 12 | Train Loss: 59.4963 | R2: -inf | Test Loss: 52.4962 | R2 0.3912
Epoch 13 | Train Loss: 5